In [1]:
import numpy as np
import matplotlib.pyplot as plt
from env import mrp
from td import *
from cvar import *
from config import *
%matplotlib inline

In [2]:
env = stopping_env()
config = config(env.nS, env.nA)


In [11]:
cvar = CVaROptimize(config)
k = 0
s = 0
env.reset()
x = env.state
while k < 10000:
    action = cvar.act(x, s)
    nx, r, terminal = env.act(action)
    s = cvar.observe(x, s, action, nx, r, k, terminal)
    k += 1
    if terminal:
        env.reset()
        x = env.state
        s = cvar.nu
print(cvar.policy)
print(compute_loss(cvar, env))

/home/ramtin/Desktop/old/CRL/cvar/cvar.py:53: RuntimeWarning: invalid value encountered in true_divide
  return np.exp(x) / np.sum(np.exp(x), axis=0)
/home/ramtin/Desktop/old/CRL/cvar/cvar.py:76: RuntimeWarning: invalid value encountered in less
  a = np.random.choice(np.arange(self.nA), p=prob)


[[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan]]
1.0


In [8]:
def compute_loss(cvar, env):
    it = 0; max_it = 10
    value_all = []
    while it < max_it:
        env.reset()
        x = env.state
        s = 0
        terminal = False
        r_ep = []
        while not terminal:
            action = cvar.act(x, s)
            nx, r, terminal = env.act(action)
            s = (s - r)/config.gamma
            r_ep.append(r)
        it += 1
        value = 0
        for r in reversed(r_ep):
            value = r + config.gamma * value
        value_all.append(value)
    return np.mean(value_all)